In [18]:
import cv2 as cv
import numpy as np

import os
import os.path as osp

from tqdm import tqdm

# SSIM

$ {\rm SSIM}(x, y) = {(2\mu_x \mu_y + c_1)(2\sigma_{xy} + c_2) \over (\mu_x^2 + \mu_y^2 + c_1)(\sigma_x^2 + \sigma_y^2 + c_2)} $

where, $L=255$ for grayscale images,

$ c_1 = (0.01L)^2 $

$ c_2 = (0.03L)^2 $ is commonly used

In [19]:
def ssim(X, Y):
    mu_x = X.mean()
    mu_y = Y.mean()
    var_x = X.var()
    var_y = Y.var()
    sigma_xy = np.cov(X.flatten(), Y.flatten())[0, 1]

    L = 255
    c1 = (0.01 * L) ** 2
    c2 = (0.03 * L) ** 2

    s1 = (2 * mu_x * mu_y + c1) / (mu_x**2 + mu_y**2 + c1)
    s2 = (2 * sigma_xy + c2) / (var_x + var_y + c2)

    SSIM = s1 * s2
    return SSIM

In [20]:
#user input
output_dir = '../inference/20250131_Base/999epoch/repaint/pair/'
gt_dir = '../DATA/VITON-HD/test/image/'
#user input

output_image_list = os.listdir(output_dir)
h_out, w_out = cv.imread(osp.join(output_dir, output_image_list[0])).shape[:2]

print('# of images:', len(output_image_list))
print('output image shape:', (w_out, h_out))

# of images: 2032
output image shape: (384, 512)


In [21]:
s_list = []

for f in tqdm(output_image_list):
    image_out = cv.imread(osp.join(output_dir, f), cv.IMREAD_GRAYSCALE)
    image_gt = cv.imread(osp.join(gt_dir, f[-12:]), cv.IMREAD_GRAYSCALE) # len('00000_00.jpg') == 12
    image_gt = cv.resize(image_gt, (w_out, h_out))
    
    s = ssim(image_out, image_gt)
    s_list.append(s)

s_arr = np.array(s_list)

100%|██████████| 2032/2032 [00:27<00:00, 74.37it/s]


In [22]:
print('------ 20241111 KCL test ------')
print('[SSIM]')

print(f'mean, std: {s_arr.mean():.3f}, {s_arr.std():.3f}')
print(f'min, max: {s_arr.min():.3f}, {s_arr.max():.3f}')

------ 20241111 KCL test ------
[SSIM]
mean, std: 0.958, 0.050
min, max: 0.569, 0.998
